In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [3]:
%cd /Users/Daniel/Desktop/UBC_MDS/525/dsci_525_group_4/data

/Users/Daniel/Desktop/UBC_MDS/525/dsci_525_group_4/data


In [20]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [21]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.41 s, sys: 5.82 s, total: 9.23 s
Wall time: 4min 23s


In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.8 s, sys: 1.36 s, total: 9.16 s
Wall time: 13 s


In [8]:
%ls -ltr figshare/

total 12061784
-rw-r--r--   1 Daniel  staff  814041183 Mar 27 15:01 data.zip
-rw-r--r--   1 Daniel  staff   95376895 Mar 27 15:03 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff   94960113 Mar 27 15:03 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff   82474546 Mar 27 15:03 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  127613760 Mar 27 15:03 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  232118894 Mar 27 15:03 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  330360682 Mar 27 15:03 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  254009247 Mar 27 15:03 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  235661418 Mar 27 15:03 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  294260911 Mar 27 15:03 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  295768615 Mar 27 15:03 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 Daniel  staff  328852

In [3]:
%%time
## here we are using a normal python way for merging the data 
fi mvnfbcdeles = glob.glob('figshare/*.csv')
df = pd.concat((pd.read_csv(file).assign(model = re.findall("/([^_]*)", file)[0]) for file in files))
df.to_csv("figshare/combined_data.csv")

CPU times: user 11min 40s, sys: 1min 12s, total: 12min 52s
Wall time: 17min 24s


In [32]:
%%time
reader = pd.read_csv("figshare/combined_data.csv", chunksize=200000, iterator=True)

CPU times: user 1.07 ms, sys: 2.51 ms, total: 3.58 ms
Wall time: 5.53 ms


In [34]:
%%time
count = 1
for chunk in reader:
    first = chunk
    if count == 1:
        break

CPU times: user 135 ms, sys: 25.3 ms, total: 160 ms
Wall time: 160 ms


In [21]:
#only use the column we want 
df = first.loc[:,'time':'model']
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
199995,1932-07-30 12:00:00,-35.439867,-33.574619,149.0625,150.9375,5.270541e-03,MPI-ESM-1-2-HAM
199996,1932-07-31 12:00:00,-35.439867,-33.574619,149.0625,150.9375,4.526228e-05,MPI-ESM-1-2-HAM
199997,1932-08-01 12:00:00,-35.439867,-33.574619,149.0625,150.9375,3.739194e-13,MPI-ESM-1-2-HAM
199998,1932-08-02 12:00:00,-35.439867,-33.574619,149.0625,150.9375,3.733396e-13,MPI-ESM-1-2-HAM


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 16000000 to 16199999
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   time           200000 non-null  object 
 1   lat_min        200000 non-null  float64
 2   lat_max        200000 non-null  float64
 3   lon_min        200000 non-null  float64
 4   lon_max        200000 non-null  float64
 5   rain (mm/day)  200000 non-null  float64
 6   model          200000 non-null  object 
dtypes: float64(5), object(2)
memory usage: 10.7+ MB


In [25]:
#change the time to date type
df['time'] = pd.to_datetime(df['time'])
df


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
16000000,1990-04-16 12:00:00,-33.0,-32.0,141.25,142.50,1.767978e+00,GFDL-CM4
16000001,1990-04-17 12:00:00,-33.0,-32.0,141.25,142.50,5.015442e-02,GFDL-CM4
16000002,1990-04-18 12:00:00,-33.0,-32.0,141.25,142.50,1.498011e-03,GFDL-CM4
16000003,1990-04-19 12:00:00,-33.0,-32.0,141.25,142.50,1.734715e-04,GFDL-CM4
16000004,1990-04-20 12:00:00,-33.0,-32.0,141.25,142.50,7.554321e-07,GFDL-CM4
...,...,...,...,...,...,...,...
16199995,1908-03-22 12:00:00,-33.0,-32.0,147.50,148.75,5.901611e-03,GFDL-CM4
16199996,1908-03-23 12:00:00,-33.0,-32.0,147.50,148.75,2.977784e-02,GFDL-CM4
16199997,1908-03-24 12:00:00,-33.0,-32.0,147.50,148.75,4.116147e-01,GFDL-CM4
16199998,1908-03-25 12:00:00,-33.0,-32.0,147.50,148.75,1.937477e-03,GFDL-CM4


In [29]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 16000000 to 16199999
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   time           200000 non-null  datetime64[ns]
 1   lat_min        200000 non-null  float64       
 2   lat_max        200000 non-null  float64       
 3   lon_min        200000 non-null  float64       
 4   lon_max        200000 non-null  float64       
 5   rain (mm/day)  200000 non-null  float64       
 6   model          200000 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 10.7+ MB
CPU times: user 19.1 ms, sys: 11.5 ms, total: 30.6 ms
Wall time: 44.1 ms


In [30]:
%%time
df.describe()

CPU times: user 27.6 ms, sys: 4.24 ms, total: 31.9 ms
Wall time: 30.8 ms


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,200000.0,200000.0,200000.000000,200000.000000,200000.000000
mean,-33.0,-32.0,144.343750,145.593750,0.909122
std,0.0,0.0,1.605841,1.605841,3.994768
min,-33.0,-32.0,141.250000,142.500000,0.000000
25%,-33.0,-32.0,142.500000,143.750000,0.000006
50%,-33.0,-32.0,143.750000,145.000000,0.001748
75%,-33.0,-32.0,146.250000,147.500000,0.107104
max,-33.0,-32.0,147.500000,148.750000,159.199368


In [31]:
%%time
df.corr()

CPU times: user 11.2 ms, sys: 3.13 ms, total: 14.3 ms
Wall time: 14 ms


<timed eval>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
lat_min,NaN,NaN,NaN,NaN,NaN
lat_max,NaN,NaN,NaN,NaN,NaN
lon_min,NaN,NaN,1.000000,1.000000,0.043832
lon_max,NaN,NaN,1.000000,1.000000,0.043832
rain (mm/day),NaN,NaN,0.043832,0.043832,1.000000


In [15]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [17]:
%%time
%%R -i df
start_time <- Sys.time()
suppressMessages(library(dplyr))
result <- df %>% count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

            model     n
1 MPI-ESM-1-2-HAM 20000
Time difference of 1.410235 secs
CPU times: user 547 ms, sys: 53.1 ms, total: 600 ms
Wall time: 1.59 s


The reason why I choosed pandas exchange is that it is really easy to use and understand.